# Data Analysis for Dublin Bus Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import datetime
import time

In [2]:
% time df = pd.read_csv('2012_one_week.csv')

Wall time: 18 s


In [3]:
df.head(5)

,Unnamed: 0,Timestamp,Line_ID,Direction,JourneyPattern_ID,Time_Frame,VehicleJourney_ID,Operator,Congestion,Longitude,Latitude,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,Time,Day,datetime
0,0,1352160002000000,41.0,0,041B0002,2012-11-05,61,SL,0,-6.264167,53.453217,-623,41008,33631,3874,1,00:00:02,Tuesday,"('00:00:02', 'Tuesday')"
1,1,1352160002000000,27.0,0,00270001,2012-11-05,4976,RD,0,-6.290833,53.319332,0,27011,33415,2355,1,00:00:02,Tuesday,"('00:00:02', 'Tuesday')"
2,2,1352160002000000,140.0,0,01400001,2012-11-05,6615,HN,0,-6.276083,53.343884,0,140010,33141,895,1,00:00:02,Tuesday,"('00:00:02', 'Tuesday')"
3,3,1352160002000000,83.0,0,083A0001,2012-11-05,7099,HN,0,-6.273733,53.419632,0,83002,40019,324,1,00:00:02,Tuesday,"('00:00:02', 'Tuesday')"
4,4,1352160002000000,16.0,0,016C0001,2012-11-05,6175,SL,0,-6.254583,53.355534,-23,16017,33633,270,1,00:00:02,Tuesday,"('00:00:02', 'Tuesday')"


In [4]:
df.drop(df.columns[[0]], axis=1, inplace=True)
df.dtypes

Timestamp              int64
Line_ID              float64
Direction              int64
JourneyPattern_ID     object
Time_Frame            object
VehicleJourney_ID      int64
Operator              object
Congestion             int64
Longitude            float64
Latitude             float64
Delay                  int64
Block_ID               int64
Vehicle_ID             int64
Stop_ID                int64
At_Stop                int64
Time                  object
Day                   object
datetime              object
dtype: object

In [5]:
df.head(3)

,Timestamp,Line_ID,Direction,JourneyPattern_ID,Time_Frame,VehicleJourney_ID,Operator,Congestion,Longitude,Latitude,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,Time,Day,datetime
0,1352160002000000,41.0,0,041B0002,2012-11-05,61,SL,0,-6.264167,53.453217,-623,41008,33631,3874,1,00:00:02,Tuesday,"('00:00:02', 'Tuesday')"
1,1352160002000000,27.0,0,00270001,2012-11-05,4976,RD,0,-6.290833,53.319332,0,27011,33415,2355,1,00:00:02,Tuesday,"('00:00:02', 'Tuesday')"
2,1352160002000000,140.0,0,01400001,2012-11-05,6615,HN,0,-6.276083,53.343884,0,140010,33141,895,1,00:00:02,Tuesday,"('00:00:02', 'Tuesday')"


In [6]:
df['Timestamp'] = df['Timestamp']//1000000

In [7]:
df.head(3)

,Timestamp,Line_ID,Direction,JourneyPattern_ID,Time_Frame,VehicleJourney_ID,Operator,Congestion,Longitude,Latitude,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,Time,Day,datetime
0,1352160002,41.0,0,041B0002,2012-11-05,61,SL,0,-6.264167,53.453217,-623,41008,33631,3874,1,00:00:02,Tuesday,"('00:00:02', 'Tuesday')"
1,1352160002,27.0,0,00270001,2012-11-05,4976,RD,0,-6.290833,53.319332,0,27011,33415,2355,1,00:00:02,Tuesday,"('00:00:02', 'Tuesday')"
2,1352160002,140.0,0,01400001,2012-11-05,6615,HN,0,-6.276083,53.343884,0,140010,33141,895,1,00:00:02,Tuesday,"('00:00:02', 'Tuesday')"


In [8]:
df.shape

(2337254, 18)

In [10]:
# defining the categorical columns
categorical_columns = df[['JourneyPattern_ID', 'VehicleJourney_ID', 'Stop_ID', 'Day']].columns

# convert the types of these columns to category
for column in categorical_columns:
    df[column] = df[column].astype('category')

In [11]:
# prepare a table of descriptive statistics for categorical features
df[categorical_columns].describe().T

,count,unique,top,freq
JourneyPattern_ID,2337224,472,046A1001,59284
VehicleJourney_ID,2337254,17323,6059,1528
Stop_ID,2337254,4563,2039,43374
Day,2337254,7,Monday,395784


In [12]:
# delete all rows wchih are not in a journey and not at stop ? 
df_groupby=df.groupby(['VehicleJourney_ID', 'JourneyPattern_ID'])

size = df_groupby.size()

keys = df_groupby.groups

In [13]:
# cut out journeys that don't have enough entries to have even left the terminus? 
df2 = df_groupby.filter(lambda x: len(x) < 13) # figure out what this line does


In [14]:
df = pd.concat([df, df2]).drop_duplicates(keep=False) 
# keep = False doesn't keep
# any of the duplicates? Should we not keep the first or last so we have one entry
# for that instance at least? 

In [15]:
df.shape

(2298944, 18)

## Add a new feature for calculating time from terminus

In [18]:
# calculate the trip time in seconds 
zscore = lambda x: (x - x.min()) # figure out what this code does
df['Trip_Time'] = df.groupby(['VehicleJourney_ID', 'JourneyPattern_ID', 'Day'])['Timestamp'].transform(zscore)


In [30]:
df.tail(5)

,Timestamp,Line_ID,Direction,JourneyPattern_ID,Time_Frame,VehicleJourney_ID,Operator,Congestion,Longitude,Latitude,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,Time,Day,datetime,Trip_Time
2337249,1352764792,38.0,0,00381003,2012-11-12,1243,PO,0,-6.261583,53.352734,-188,38003,36045,265,1,23:59:52,Monday,"('23:59:52', 'Monday')",2089
2337250,1352764792,37.0,0,00371002,2012-11-12,2115,PO,0,-6.260700,53.347198,-249,37003,33578,313,1,23:59:52,Monday,"('23:59:52', 'Monday')",1325
2337251,1352764792,46.0,0,046A0001,2012-11-12,7345,D2,0,-6.231650,53.318817,0,46011,36020,2039,1,23:59:52,Monday,"('23:59:52', 'Monday')",4928
2337252,1352764796,16.0,0,00161001,2012-11-12,5652,SL,0,-6.218033,53.425232,-488,16004,33425,7347,1,23:59:56,Monday,"('23:59:56', 'Monday')",4371
2337253,1352764798,7.0,0,00070001,2012-11-12,6963,D1,0,-6.138767,53.276184,-128,7014,43007,2040,1,23:59:58,Monday,"('23:59:58', 'Monday')",1319


In [20]:
df_15744=df.loc[df['VehicleJourney_ID'] == 15744]

In [21]:
df_15744[categorical_columns].describe().T

,count,unique,top,freq
JourneyPattern_ID,77,1,07471001,77
VehicleJourney_ID,77,1,15744,77
Stop_ID,77,6,4319,41
Day,77,1,Sunday,77


## Check the data for one VehicleJourney_ID 6187

In [22]:
df_6187 = df.loc[df.VehicleJourney_ID==6187]

In [23]:
df_6187[categorical_columns].describe().T

,count,unique,top,freq
JourneyPattern_ID,397,1,00270001,397
VehicleJourney_ID,397,1,6187,397
Stop_ID,397,74,2355,57
Day,397,5,Thursday,97


In [24]:
pd.unique(df_6187.JourneyPattern_ID.ravel()) # what does ravel do? 

array(['00270001'], dtype=object)

In [25]:
pd.unique(df_6187.Day.ravel())

array(['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Monday'], dtype=object)

In [27]:
df_6187_Mon = df_6187.loc[df_6187['Day'] == 'Monday']

In [28]:
df_6187_Mon[categorical_columns].describe().T

,count,unique,top,freq
JourneyPattern_ID,80,1,00270001,80
VehicleJourney_ID,80,1,6187,80
Stop_ID,80,50,4595,17
Day,80,1,Monday,80


In [31]:
df_6187_Mon.shape

(80, 19)

In [32]:
df_6187_Mon

,Timestamp,Line_ID,Direction,JourneyPattern_ID,Time_Frame,VehicleJourney_ID,Operator,Congestion,Longitude,Latitude,Delay,Block_ID,Vehicle_ID,Stop_ID,At_Stop,Time,Day,datetime,Trip_Time
2155470,1352733307,27.0,0,00270001,2012-11-12,6187,RD,0,-6.172567,53.402115,0,27006,33252,4595,1,15:15:07,Monday,"('15:15:07', 'Monday')",0
2155599,1352733328,27.0,0,00270001,2012-11-12,6187,RD,0,-6.172567,53.402115,0,27006,33252,4595,1,15:15:28,Monday,"('15:15:28', 'Monday')",21
2155711,1352733346,27.0,0,00270001,2012-11-12,6187,RD,0,-6.172567,53.402115,0,27006,33252,4595,1,15:15:46,Monday,"('15:15:46', 'Monday')",39
2155833,1352733367,27.0,0,00270001,2012-11-12,6187,RD,0,-6.172567,53.402115,0,27006,33252,4595,1,15:16:07,Monday,"('15:16:07', 'Monday')",60
2155974,1352733389,27.0,0,00270001,2012-11-12,6187,RD,0,-6.172567,53.402115,0,27006,33252,4595,1,15:16:29,Monday,"('15:16:29', 'Monday')",82
2156083,1352733406,27.0,0,00270001,2012-11-12,6187,RD,0,-6.172567,53.402115,0,27006,33252,4595,1,15:16:46,Monday,"('15:16:46', 'Monday')",99
2156345,1352733449,27.0,0,00270001,2012-11-12,6187,RD,0,-6.172567,53.402115,0,27006,33252,4595,1,15:17:29,Monday,"('15:17:29', 'Monday')",142
2156489,1352733467,27.0,0,00270001,2012-11-12,6187,RD,0,-6.172567,53.402115,0,27006,33252,4595,1,15:17:47,Monday,"('15:17:47', 'Monday')",160
2156583,1352733488,27.0,0,00270001,2012-11-12,6187,RD,0,-6.172567,53.402115,0,27006,33252,4595,1,15:18:08,Monday,"('15:18:08', 'Monday')",181
2156706,1352733506,27.0,0,00270001,2012-11-12,6187,RD,0,-6.172567,53.402115,0,27006,33252,4595,1,15:18:26,Monday,"('15:18:26', 'Monday')",199


## Data Analysis Based on H1 from Data Analytics Module

In [33]:
df.shape

(2298944, 19)